In [1]:
import os
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

In [3]:
torch.cuda.is_available()
torch.cuda.get_device_name()

'Tesla T4'

In [ ]:
# Load the LLAMA2 model and tokenizer
model_name = "path/to/llama2_model"  # Replace with the path to your LLAMA2 model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Define the prompt
prompt = """
System: You are a SQL generation assistant.
User: Please generate an SQL query to retrieve all orders placed in the last month.
User: The database schema includes tables 'orders' and 'order_date.'
"""

# Tokenize the prompt
input_ids = tokenizer.encode(prompt, return_tensors="pt", truncation=True)

# Generate an SQL query
output = model.generate(input_ids, max_length=150, num_return_sequences=1, no_repeat_ngram_size=2)

# Decode and print the SQL query
sql_query = tokenizer.decode(output[0], skip_special_tokens=True)
print("Generated SQL Query:", sql_query)